In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import folium 
import random
import numpy as np
from pyomo.environ import *

In [3]:
def load_distance_time_matrix(path, N):
    data= pd.read_csv(path)
    distance = np.zeros((N,N))
    time = np.zeros((N,N))
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen-1, destino-1] = float(data.iloc[i, 2])
        time[origen-1, destino-1] = float(data.iloc[i, 3])
    return distance, time

def load_vehicles(path):
    data= pd.read_csv(path)
    N = len(data)
    vehicles = {}
    for i in range(N):
        id = int(data.iloc[i, 0])
        capacity = int(data.iloc[i, 1])
        ran = float(data.iloc[i, 2])
        vehicles[id] = (capacity, ran)
    
    return vehicles

def load_demand(clientsPath):
    data= pd.read_csv(clientsPath)
    N = len(data)
    demand_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        demand = float(data.iloc[i, 2])
        demand_dic[id] = demand
    
    return demand_dic


def load_capacity(depositPath):
    data= pd.read_csv(depositPath)
    N = len(data)
    capacity_dic = {}
    for i in range(N):
        id = int(data.iloc[i, 1])
        capacity = float(data.iloc[i, 3])
        capacity_dic[id] = capacity
    
    return capacity_dic

def load_distance_time_dic(path):
    data= pd.read_csv(path)
    distance = {}
    time = {}
    
    for i in range(len(data)):
        origen = int(data.iloc[i, 0])
        destino = int(data.iloc[i, 1])
        distance[origen, destino] = float(data.iloc[i, 2])
        time[origen, destino] = float(data.iloc[i, 3])
    
    return distance, time

def load_coordinates(depotsPath, clientsPath):
    coord = {}
    depot= pd.read_csv(depotsPath)
    client= pd.read_csv(clientsPath)
    
    for i in range(len(depot)):
        id = int(depot.iloc[i, 1])
        lat = float(depot.iloc[i, 3])
        long = float(depot.iloc[i, 2])
        coord[id] = [lat, long]

    for j in range(len(client)):
        id = int(client.iloc[j, 1])
        lat = float(client.iloc[j, 4])
        long = float(client.iloc[j, 3])
        coord[id] = [lat, long]
    
    return coord
        
        

In [4]:
#Datos artificiales

np.random.seed(42)
n = 25  # 24 clientes + 1 depósito

dist_matrix_np, time_matrix = load_distance_time_matrix('..\datos\Caso_Base\casoBase.csv', 25)
# Convertir a diccionario {(i,j): distancia}
distancia = {(i, j): dist_matrix_np[i][j]/1000 for i in range(n) for j in range(n)}

"""
vehiculos = {
  1: (130, 170),
  2: (140, 200),
  3: (120, 180),
  4: (100, 90),
  5: (70, 100),
  6: (55, 170),
  7: (110, 150),
  8: (114, 140)
}

demanda = {
  0:0, 1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15,
  11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12,
  19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11
}
"""

vehiculos = load_vehicles('../Datos/Caso_Base/vehicles.csv')
oferta = load_capacity('..\Datos\Caso_Base\depots.csv')
demanda = load_demand('..\Datos\Caso_Base\clients.csv')
coord = load_coordinates('..\Datos\Caso_Base\depots.csv', '..\Datos\Caso_Base\clients.csv')
distancia_dic,_ = load_distance_time_dic('..\Datos\Caso_Base\casoBase.csv')

print(f"Distancias: {distancia_dic}")
print(f"Vehiculos: {vehiculos}")
print(f"Oferta: {oferta}")
print(f"Demanda: {demanda}")
print(f"Coordenadas: {coord}")


print(coord)

Distancias: {(1, 2): 27144.3, (1, 3): 17677.1, (1, 4): 13979.6, (1, 5): 26650.8, (1, 6): 22583.7, (1, 7): 18562.5, (1, 8): 24765.9, (1, 9): 23929.4, (1, 10): 27835.9, (1, 11): 33651.8, (1, 12): 31029.4, (1, 13): 16124.7, (1, 14): 21612.2, (1, 15): 10387.4, (1, 16): 27040.2, (1, 17): 33963.1, (1, 18): 26541.3, (1, 19): 20053.2, (1, 20): 21278.2, (1, 21): 13411.2, (1, 22): 27869.5, (1, 23): 24262.6, (1, 24): 11364.8, (1, 25): 30809.8, (2, 1): 30809.9, (2, 3): 14256.3, (2, 4): 19400.3, (2, 5): 1130.9, (2, 6): 12475.1, (2, 7): 11416.7, (2, 8): 16851.4, (2, 9): 13820.8, (2, 10): 7509.8, (2, 11): 7110.4, (2, 12): 10890.2, (2, 13): 24445.1, (2, 14): 7973.9, (2, 15): 20142.5, (2, 16): 1255.4, (2, 17): 7421.8, (2, 18): 4283.4, (2, 19): 9819.6, (2, 20): 15281.3, (2, 21): 23046.5, (2, 22): 21147.7, (2, 23): 5482.0, (2, 24): 21239.2, (2, 25): 4880.9, (3, 1): 18222.2, (3, 2): 12672.2, (3, 4): 6812.6, (3, 5): 12178.7, (3, 6): 14735.1, (3, 7): 8279.5, (3, 8): 8286.5, (3, 9): 16080.8, (3, 10): 15410.7

In [5]:
# Crear el modelo
model = ConcreteModel()

# Conjuntos e índices
nodos = [1] + list(demanda.keys())  # El nodo 1 es el depósito
model.N = Set(initialize=nodos)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=[1])  # Nodo depósito

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i por vehículo v

# Parámetros

# Asumiendo que tus nodos van del 1 al n
offset = 1
dist_modificada = {(i+offset, j+offset): v for (i, j), v in distancia.items()}
model.dist = Param(model.N, model.N, initialize=dist_modificada, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función Objetivo
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Flujo conservado por vehículo en cada cliente
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Salida del depósito por vehículo es máximo 1
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[1, j, v] for j in model.N if j != 1) == 1  # El nodo depósito es 1
    )

# Capacidad del vehículo no debe excederse
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )

# Valor inicial de carga en el depósito es 1
model.initial_load = ConstraintList()
for v in model.V:
    model.initial_load.add(model.u[1, v] == 1)

# La carga nunca puede superar la capacidad del vehículo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo no debe excederse
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)

# Resolver el modelo
opt = SolverFactory('glpk')
results = opt.solve(model, tee=False)

# Resultados
print("Valor de la función objetivo:", model.obj())
# Mostrar capacidad utilizada por ruta
routes = {}
for v in model.V:
    print(f"Vehículo {v}:")
    routes[v] = []
    for i in model.N:
        for j in model.N:
            if i != j and model.x[i, j, v].value > 0.5:  # Solo si la ruta es tomada
                origen = coord[i]
                destino = coord[j]
                
                if origen not in routes[v]:
                    routes[v].append(origen)
                if destino not in routes[v]:
                    routes[v].append(destino)
                
                if i in model.C:
                    capacidad_utilizada = model.demand[i] * model.x[i, j, v].value
                    print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
                else:
                    print(f"  Ruta {i} -> {j} (desde depósito)")

Valor de la función objetivo: 315.67189999999994
Vehículo 1:
  Ruta 1 -> 4 (desde depósito)
  Ruta 2 -> 16: Carga transportada = 13.0
  Ruta 4 -> 1: Carga transportada = 12.0
  Ruta 5 -> 2: Carga transportada = 15.0
  Ruta 10 -> 12: Carga transportada = 20.0
  Ruta 12 -> 10: Carga transportada = 17.0
  Ruta 16 -> 5: Carga transportada = 17.0
  Ruta 18 -> 25: Carga transportada = 25.0
  Ruta 25 -> 18: Carga transportada = 11.0
Vehículo 2:
  Ruta 1 -> 3 (desde depósito)
  Ruta 3 -> 1: Carga transportada = 15.0
  Ruta 8 -> 22: Carga transportada = 17.0
  Ruta 14 -> 23: Carga transportada = 21.0
  Ruta 22 -> 8: Carga transportada = 14.0
  Ruta 23 -> 14: Carga transportada = 18.0
Vehículo 3:
  Ruta 1 -> 21 (desde depósito)
  Ruta 21 -> 1: Carga transportada = 15.0
Vehículo 4:
  Ruta 1 -> 13 (desde depósito)
  Ruta 13 -> 1: Carga transportada = 12.0
Vehículo 5:
  Ruta 1 -> 24 (desde depósito)
  Ruta 24 -> 1: Carga transportada = 15.0
Vehículo 6:
  Ruta 1 -> 15 (desde depósito)
  Ruta 11 -> 1

In [ ]:
# Crear el modelo
model = ConcreteModel()

# Conjuntos e índices
nodos = [1] + list(demanda.keys())  # El nodo 1 es el depósito
model.N = Set(initialize=nodos)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=[1])  # Nodo depósito

# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.y = Var(model.C, model.V, domain=Binary)  # Cliente i es atendido por vehículo v
model.u = Var(model.C, model.V, bounds=(0, None), domain=NonNegativeReals)  # Variable para eliminación de subtours
model.used = Var(model.V, domain=Binary)  # Indica si el vehículo v es utilizado

# Parámetros
# Usar la matriz de distancias completa
model.dist = Param(model.N, model.N, initialize=distancia_dic, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función Objetivo: Minimizar la distancia total recorrida
def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

# R1: Cada cliente es visitado exactamente una vez
def visit_once_rule(m, j):
    if j in m.C:
        return sum(m.x[i, j, v] for i in m.N if i != j for v in m.V) == 1
    return Constraint.Skip
model.visit_once = Constraint(model.N, rule=visit_once_rule)

# R2: El vehículo que llega a un cliente también debe salir de él (continuidad de flujo)
def flow_conservation_rule(m, j, v):
    if j in m.C:
        return sum(m.x[i, j, v] for i in m.N if i != j) == sum(m.x[j, i, v] for i in m.N if i != j)
    return Constraint.Skip
model.flow_conservation = Constraint(model.N, model.V, rule=flow_conservation_rule)

# R3: Cada vehículo sale del depósito a lo sumo una vez
def depot_departure_rule(m, v):
    return sum(m.x[1, j, v] for j in m.C) <= 1
model.depot_departure = Constraint(model.V, rule=depot_departure_rule)

# R4: Un vehículo que sale del depósito debe regresar al depósito
def depot_return_rule(m, v):
    return sum(m.x[1, j, v] for j in m.C) == sum(m.x[i, 1, v] for i in m.C)
model.depot_return = Constraint(model.V, rule=depot_return_rule)

# R5: Relacionar si un cliente es atendido por un vehículo
def served_by_rule(m, i, v):
    if i in m.C:
        return sum(m.x[j, i, v] for j in m.N if j != i) == m.y[i, v]
    return Constraint.Skip
model.served_by = Constraint(model.N, model.V, rule=served_by_rule)

# R6: La capacidad del vehículo no debe excederse
def capacity_rule(m, v):
    return sum(m.demand[i] * m.y[i, v] for i in m.C) <= m.cap[v]
model.capacity = Constraint(model.V, rule=capacity_rule)

# R7: La distancia recorrida por un vehículo no debe exceder su rango
def range_rule(m, v):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j) <= m.range[v] * 1000  # Convertir a metros
model.range_constraint = Constraint(model.V, rule=range_rule)

# R8: Si un vehículo hace cualquier ruta, se considera usado
def used_vehicle_rule(m, v):
    return sum(m.x[1, j, v] for j in m.C) == m.used[v]
model.used_vehicle = Constraint(model.V, rule=used_vehicle_rule)

# R9: Eliminar subtours usando restricciones MTZ
model.subtour_elimination = ConstraintList()
for v in model.V:
    for i in model.C:
        for j in model.C:
            if i != j:
                model.subtour_elimination.add(
                    model.u[j, v] >= model.u[i, v] + model.demand[j] - model.cap[v] * (1 - model.x[i, j, v])
                )
                
# R10: Inicializar valores de u para cuando un cliente es el primero en ser visitado desde el depósito
def u_init_rule(m, i, v):
    if i in m.C:
        return m.u[i, v] >= m.demand[i] - m.cap[v] * (1 - m.x[1, i, v])
    return Constraint.Skip
model.u_init = Constraint(model.N, model.V, rule=u_init_rule)

# R11: Limitar u para clientes no visitados por un vehículo
def u_limit_rule(m, i, v):
    if i in m.C:
        return m.u[i, v] <= m.cap[v] * m.y[i, v]
    return Constraint.Skip
model.u_limit = Constraint(model.N, model.V, rule=u_limit_rule)

# R12: No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)

# Resolver el modelo con un límite de tiempo más generoso
solver = SolverFactory('gurobi')
# Aumentar el tiempo límite a 30 minutos (1800 segundos)
results = solver.solve(model, tee=True, options={'TimeLimit': 600, 'MIPGap': 0.05})
cond = results.solver.termination_condition

# Acepta resultados aunque no sean óptimos
if cond in [TerminationCondition.optimal, TerminationCondition.feasible]:
    print("✔️ Se encontró una solución (óptima o factible).")
    # Ahora es seguro acceder a los valores
    print("Valor de la función objetivo:", value(model.obj))
    
    # Mostrar capacidad utilizada por ruta
    routes = {}
    for v in model.V:
        print(f"Vehículo {v}:")
        routes[v] = []
        for i in model.N:
            for j in model.N:
                if i != j and model.x[i, j, v].value > 0.5:  # Solo si la ruta es tomada
                    origen = coord[i]
                    destino = coord[j]
                    
                    if origen not in routes[v]:
                        routes[v].append(origen)
                    if destino not in routes[v]:
                        routes[v].append(destino)
                    
                    if i in model.C:
                        capacidad_utilizada = model.demand[i] * model.x[i, j, v].value
                        print(f"  Ruta {i} -> {j}: Carga transportada = {capacidad_utilizada}")
                    else:
                        print(f"  Ruta {i} -> {j} (desde depósito)")

else:
    print("No se encontró una solución factible.")
    print("Condición de terminación:", cond)
    print("Estado del solver:", results.solver.status)


Read LP format model from file C:\Users\aleja\AppData\Local\Temp\tmp8g9tlhsp.pyomo.lp
Reading time = 0.04 seconds
x1: 5448 rows, 5392 columns, 38608 nonzeros
Set parameter TimeLimit to value 600
Set parameter MIPGap to value 0.05
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10300H CPU @ 2.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  600
MIPGap  0.05

Optimize a model with 5448 rows, 5392 columns and 38608 nonzeros
Model fingerprint: 0x8d732007
Variable types: 192 continuous, 5200 integer (5200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+04]
  Objective range  [7e+02, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+05]
Presolve removed 208 rows and 208 columns
Presolve time: 0.08s
Presolved: 5240 rows, 5184 columns, 29368 nonzeros
Variable types: 192 continuous, 4992 intege

## Validación de la solución

Para validar la solución, se va a revisar que se cumplan las restricciones del problema y se generará el archivo respectivo para que se valide de forma rápida la solución que se obtuvo. Este archivo se encuentra dentro de la carpeta de `resultados_docs`.

### Validación analítica

### Archivo de validación

In [5]:
m = folium.Map(
    location=[4.743359, -74.153536],
    zoom_start=11,
    tiles='Cartodb Positron' 
)

number_of_colors = 9

colors = ['blue', 'green', 'cyan', 'magenta','olive', 'blue', 'orange', 'purple','red' ]
icons = ['blue', 'green', 'lightblue', 'pink','lightgreen', 'blue', 'orange', 'darkpurple','red' ]

for route in routes.keys():
    if len(routes[route]) != 0 :  
        folium.PolyLine(
            routes[route],
            color=colors[route],
            weight=5,
            opacity=0.7,
            tooltip='Vehículo ' + str(route)
        ).add_to(m)

        folium.Marker(routes[route][0], popup="Inicio", icon=folium.Icon(color='black')).add_to(m)
        folium.Marker(routes[route][-1], popup="Llegada V" + str(route), icon=folium.Icon(color=icons[route])).add_to(m)
m